# V3 Narasipal Scraping
Links only, no other attributes

In [6]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from seleniumbase import Driver
import csv
import time
import sys
sys.path.append("/Users/salmadanu/Desktop/Skripsi/skripsi-env/skripsienv/lib/python3.9/site-packages")
import undetected_chromedriver as uc

# Kompas

In [ ]:
def initialize_driver_kompas(pagenum):
    driver = Driver(uc=True)
    driver.get(f"https://www.kompas.com/tag/palestina?page={pagenum}")
    return driver

In [ ]:
def scrape_links_from_page_kompas(driver):
    time.sleep(5)
    my_page = driver.page_source
    my_html = BeautifulSoup(my_page, "html.parser")
    card_containers = my_html.find_all('div', class_='article__list clearfix')

    nav_links = []
    for container in card_containers:
        link_link = container.find('a', href=True)
        href = link_link['href'] if link_link else None
        if href:
            nav_links.append(href)
    return nav_links

In [ ]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)

# Combine CSV Kompas

In [ ]:
import os
import pandas as pd

file_path = "/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/kompas_new"
csv_files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith('.csv')]

dataframes = []
for f in csv_files:
    df = pd.read_csv(f)
    if 'article_link' in df.columns:
        df.rename(columns={'article_link': 'url'}, inplace=True)
    dataframes.append(df)

kompas_master = pd.concat(dataframes, ignore_index=True)

output_path = "/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/master_csv/kompas_master.csv"
kompas_master.to_csv(output_path, index=False)

print(f"Master CSV saved to {output_path}")

In [ ]:
all_nav_links = []
try:
    for page_num in range(6, 70): # Halaman terakhir ga include
        print(f"Scraping links from page {page_num}...")
        driver.get(f"https://www.kompas.com/tag/palestina?page={page_num}")
        page_links = scrape_links_from_page_kompas(driver)
        all_nav_links.extend(page_links)
except Exception as e:
    print(f"Error while scraping links: {e}")
finally:
    driver.quit()

if all_nav_links:
    output_file = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/kompas_new/kompas_6-69.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["article_link"])
    for link in all_nav_links:
        writer.writerow([link])

print(f"Links written to {output_file}")

# Republika

In [ ]:
def initialize_driver_republika(linknum):
    driver = Driver(uc=True)
    driver.get(f"https://republika.co.id/tag/palestina/{linknum}")
    return driver

In [ ]:
def scrape_articles_from_page_republika(driver):
    time.sleep(5)
    my_page = driver.page_source
    my_html = BeautifulSoup(my_page, "html.parser")
    articles = my_html.find_all('li', class_='list-group-item list-border conten1')
    
    scraped_data = []
    for article in articles:
        link_tag = article.find('a', href=True)
        url = link_tag['href'] if link_tag else None
        
        date_span = article.find('div', class_='date')
        date_time = date_span.text.split(' - ')[-1] if date_span else None
        
        title_tag = article.find('h3').find('span') if article.find('h3') else None
        title = title_tag.text.strip() if title_tag else None
        
        if url and date_time and title:
            scraped_data.append([url, date_time, title])
    
    return scraped_data

In [ ]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)

In [ ]:
all_articles = []
try:
    for link_num in range(75, 225, 15):  # Halaman terakhir ga include, tambahin step 15
        print(f"Scraping links from page {link_num}...")
        driver.get(f"https://republika.co.id/tag/palestina/{link_num}")
        page_articles = scrape_articles_from_page_republika(driver)
        all_articles.extend(page_articles)
except Exception as e:
    print(f"Error while scraping links: {e}")
finally:
    driver.quit()

if all_articles:
    output_file = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/republika/republika_6-15.csv'
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["article_link", "date_time", "article_title"])
        for article in all_articles:
            writer.writerow(article)

print(f"Articles written to {output_file}")

# CNN

In [ ]:
def initialize_driver_cnnindonesia(pagenum):
    driver = Driver(uc=True)
    driver.get(f"https://www.cnnindonesia.com/tag/palestina?page={pagenum}")
    return driver

In [ ]:
def scrape_links_from_page_cnnindonesia(driver):
    time.sleep(5)
    my_page = driver.page_source
    my_html = BeautifulSoup(my_page, "html.parser")

    # Find all div containers with the target class
    card_containers = my_html.find_all('div', class_='flex flex-col gap-5')

    nav_links = []
    for container in card_containers:
        links = container.find_all('a', href=True)  # Get all <a> tags inside the container
        for link in links:
            href = link['href']
            if href.startswith("https://www.cnnindonesia.com/"):  # Ensure it's an article link
                nav_links.append(href)

    return list(set(nav_links))  # Remove duplicates


In [ ]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)

In [ ]:
all_nav_links = []
try:
    for page_num in range(18, 50): # Halaman terakhir ga include
        print(f"Scraping links from page {page_num}...")
        driver.get(f"https://www.cnnindonesia.com/tag/palestina?page={page_num}")
        page_links = scrape_links_from_page_cnnindonesia(driver)
        all_nav_links.extend(page_links)
except Exception as e:
    print(f"Error while scraping links: {e}")
finally:
    driver.quit()

if all_nav_links:
    output_file = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/cnnindonesia/cnn_18-49.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["article_link"])
    for link in all_nav_links:
        writer.writerow([link])

print(f"Links written to {output_file}")

# Detik

In [ ]:
def initialize_driver_detik(pagenum):
    driver = Driver(uc=True)
    driver.get(f"https://www.detik.com/tag/palestina/?sortby=time&page={pagenum}")
    return driver

In [ ]:
def scrape_articles_from_page_detik(driver):
    time.sleep(5)
    my_page = driver.page_source
    my_html = BeautifulSoup(my_page, "html.parser")
    
    # Find all articles
    articles = my_html.find_all('article')

    scraped_data = []
    for article in articles:
        link_tag = article.find('a', href=True)
        url = link_tag['href'] if link_tag else None
        
        date_span = article.find('span', class_='date')
        date_time = date_span.text.split(', ')[-1] if date_span else None
        
        title_tag = article.find('h2', class_='title')
        title = title_tag.text.strip() if title_tag else None

        if url and date_time and title:
            scraped_data.append([url, date_time, title])
    
    return scraped_data

In [ ]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)

In [ ]:
all_articles = []
try:
    for page_num in range(535, 546):  # Halaman terakhir ga include
        print(f"Scraping links from page {page_num}...")
        driver.get(f"https://www.detik.com/tag/palestina/?sortby=time&page={page_num}")
        page_articles = scrape_articles_from_page_detik(driver)
        all_articles.extend(page_articles)
except Exception as e:
    print(f"Error while scraping links: {e}")
finally:
    driver.quit()

if all_articles:
    output_file = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/detik/detik_544-545.csv'
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["article_link", "date_time", "article_title"])
        for article in all_articles:
            writer.writerow(article)

print(f"Articles written to {output_file}")